In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.misc 
import scipy.io
#from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline  
print ("Packages loaded.")

Packages loaded.


https://github.com/sjchoi86/Tensorflow-101/blob/master/notebooks/cnn_finetune_vgg.ipynb
위 링크를 따라 함


In [2]:
cwd = '/home/nips/data/design/'
loadpath = cwd + "/npz_data/design_data.npz"
l = np.load(loadpath)

# Show Files
print (l.files)

['trainlabel', 'imgsize', 'trainimg', 'testimg', 'testlabel', 'use_gray', 'categories']


In [3]:
trainimg   = l['trainimg']
trainlabel = l['trainlabel']
testimg    = l['testimg']
testlabel  = l['testlabel']
ntrain     = trainimg.shape[0]
nclass     = trainlabel.shape[1]
dim        = trainimg.shape[1]
ntest      = testimg.shape[0]

print ("%d train images loaded" % (ntrain))
print ("%d test images loaded"  % (ntest))
print ("%d dimensional input"   % (dim))
print ("%d classes"             % (nclass))
print ("shape of 'trainimg' is %s" % (trainimg.shape,))
print ("shape of 'testimg' is %s" % (testimg.shape,))

16000 train images loaded
4000 test images loaded
4096 dimensional input
20 classes
shape of 'trainimg' is (16000, 4096)
shape of 'testimg' is (4000, 4096)


In [4]:
trainimg_tensor = np.ndarray((ntrain, 4096))
for i in range(ntrain):
    currimg = trainimg[i, :]
#    currimg = np.reshape(currimg, [112, 112, 3])
    trainimg_tensor[i, :] = currimg
print ("shape of trainimg_tensor is %s" % (trainimg_tensor.shape,))

testimg_tensor = np.ndarray((ntest, 4096))
for i in range(ntest):
    currimg = testimg[i, :]
#    currimg = np.reshape(currimg, [112, 112, 3])
    testimg_tensor[i, :] = currimg
print ("shape of testimg_tensor is %s" % (testimg_tensor.shape,))

shape of trainimg_tensor is (16000, 4096)
shape of testimg_tensor is (4000, 4096)


In [11]:
data = scipy.io.loadmat(loadpath)
'meta' in data

ValueError: Unknown mat file type, version 0, 0

In [6]:
def net(data_path, input_image):
    layers = (
        'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',
        'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',
        'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
        'relu3_3', 'conv3_4', 'relu3_4', 'pool3',
        'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
        'relu4_3', 'conv4_4', 'relu4_4', 'pool4',
        'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
        'relu5_3', 'conv5_4', 'relu5_4'
    )
    data = scipy.io.loadmat(data_path)
#    mean = data['normalization'][0][0][0]
    mean = data['meta'][0][0][2][0][0][2]
    mean_pixel = np.mean(mean, axis=(0, 1))
    weights = data['layers'][0]
    net = {}
    current = input_image
    for i, name in enumerate(layers):
        kind = name[:4]
        if kind == 'conv':
#            kernels, bias = weights[i][0][0][0][0]
            kernels, bias = weights[i][0][0][2][0]
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current)
        net[name] = current

    assert len(net) == len(layers)
    return net, mean_pixel, layers
print ("Network for VGG ready")

Network for VGG ready


In [7]:
def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)
def _pool_layer(input):
    return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
            padding='SAME')
def preprocess(image, mean_pixel):
    return image - mean_pixel
def unprocess(image, mean_pixel):
    return image + mean_pixel
def imread(path):
    return scipy.misc.imread(path).astype(np.float)
def imsave(path, img):
    img = np.clip(img, 0, 255).astype(np.uint8)
    scipy.misc.imsave(path, img)
print ("Functions for VGG ready")

Functions for VGG ready


https://github.com/itswyri/tensorflow_examples/blob/master/cam_vgg.ipynb
위 링크를 참고하여 파라미터를 바꿔줘라. 명령어가 업데이트되면서 변경되었다,.
https://github.com/sjchoi86/Tensorflow-101/blob/master/notebooks/use_vgg.ipynb

In [8]:
VGG_PATH = cwd + "pretrained_model/imagenet-vgg-verydeep-19.mat"

In [9]:
VGG_PATH

'/home/nips/data/design/pretrained_model/imagenet-vgg-verydeep-19.mat'

In [10]:
with tf.Graph().as_default(), tf.Session() as sess:
    with tf.device("/gpu:0"):
        img_placeholder = tf.placeholder(tf.float32, shape=(None, 64, 64, 3))
        net_val, mean_pixel = net(VGG_PATH, img_placeholder)
        train_features = net_val['relu5_4'].eval(feed_dict={img_placeholder: trainimg_tensor})
        test_features = net_val['relu5_4'].eval(feed_dict={img_placeholder: testimg_tensor})

ValueError: too many values to unpack

In [ ]:
print ("TYPE OF 'train_features' IS %s" % (type(train_features)))
print ("SHAPE OF 'train_features' IS %s" % (train_features.shape,))
print ("TYPE OF 'test_features' IS %s" % (type(test_features)))
print ("SHAPE OF 'test_features' IS %s" % (test_features.shape,))
print("PREPROCESSING DONE")